In [ ]:
!pip install flask-ngrok

In [ ]:
!pwd

/content


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

@app.route("/")
def predict():

  return render_template("submit_form.html")

@app.route('/result', methods=['POST'])
def result():

  data = request.form

  message = ''
  message += "<hl>House Price</h1>"

  for k, v in data.items():
    print(k, v)
    message += k + ": " + v + "</br>"

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9c3d5d0ed0f6.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler

In [ ]:
x = pd.read_csv('x.csv')

with open('y.npy', 'rb') as f:
  y = np.load(f)

In [ ]:
scaled_x.shape, scaled_y.shape

((1460, 8), (1460, 1))

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=scaled_x.shape[-1]),
        layers.Dense(96, activation='relu'),
        layers.Dense(48, activation='relu'),
        layers.Dense(1)
    ]
)

model.compile(loss='mse', optimizer = 'adam')

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)
model.fit(scaled_x, scaled_y,
          batch_size=2, epochs=150,
          callbacks=[early_stopping_callback], validation_split=0.05)

Epoch 1/150
694/694 [==============================] - 2s 2ms/step - loss: 0.0047 - val_loss: 0.0029
Epoch 2/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0033 - val_loss: 0.0032
Epoch 3/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0033 - val_loss: 0.0014
Epoch 4/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0027 - val_loss: 0.0018
Epoch 5/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0029 - val_loss: 0.0017
Epoch 6/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0024 - val_loss: 0.0019
Epoch 7/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0027 - val_loss: 0.0014
Epoch 8/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0026 - val_loss: 0.0014
Epoch 9/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0029 - val_loss: 0.0024
Epoch 10/150
694/694 [==============================] - 1s 2ms/step - loss: 0.0029 - val_lo

In [ ]:
pred = reconstructed_model.predict(scaled_x[:5])
pred = y_min_max_scaler.inverse_transform(pred)

In [ ]:
pred.shape

(5, 1)

# **Save MLP model**

In [ ]:
model.save("mlp_v0.1.h5")

In [ ]:
reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

x = x[['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'LotShape_rank']]
x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(x)
scaled_x = x_min_max_scaler.transform(x)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(y)
scaled_y = y_min_max_scaler.transform(y)

reconstructed_model = keras.models.load_model("mlp_v0.1.h5")

app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

def preprocess_data(data):
  # return np.zeros((1, 8)) # dummy data

  """
  Dictionary --> np.array (1,8)

  OverallQual 2
  GrLivArea 5000
  GarageCars 2
  GarageArea 480
  TotalBsmtSF 991
  1stFlrSF 1087
  FullBath 2
  LotShape IR3 --> 1, 2, 3, 4
  """

  #Scale Normalization

  x = []
  for k, v in data.items():
    if k == "LotShape":
      if v == "Reg":
        x.append(4)
      elif v == "IR3":
        x.append(3)
      elif v == "IR2":
        x.append(2)
      elif v == "IR1":
        x.append(1)
    else:
      x.append(float(v))

  x = np.array(x)
  x = x.reshape((1, -1))

  #min max scaling
  scaled_x = x_min_max_scaler.transform(x)
  print(scaled_x.shape)

  return scaled_x


@app.route("/")
def predict():

  return render_template("submit_form.html")

@app.route('/result', methods=['POST'])
def result():

  data = request.form

  message = ''
  message += "<hl>House Price</h1>"

  for k, v in data.items():
    print(k, v)
    message += k + ": " + v + "</br>"

  x = preprocess_data(data)
  pred = model.predict(x)
  pred = y_min_max_scaler.inverse_transform(pred)

  message += "</br>"
  message += "Predicted price: " + str(pred[0][0])

  return message

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2d831274bf1f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Aug/2021 13:59:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 13:59:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Aug/2021 13:59:04] "POST /result HTTP/1.1" 200 -


OverallQual 10
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1
(1, 8)


127.0.0.1 - - [11/Aug/2021 13:59:14] "POST /result HTTP/1.1" 200 -


OverallQual 5
GrLivArea 1464
GarageCars 2
GarageArea 480
TotalBsmtSF 991
1stFlrSF 1087
FullBath 2
LotShape IR1
(1, 8)
